In [ ]:
# 라이브러리 불러오기
import threading
import cv2
import math
import numpy as np
import matplotlib.pyplot as plt
import torch

from time import sleep
# from yolov3.detect_ceustom import run_yolo
from pose_ArUCo.pose_estimation import run_aruco
from pose_ArUCo.detect_aruco_images import detect_marker
from dobot_api import DobotApiDashboard, DobotApi, DobotApiMove, MyType
from rotate import transform_cam_to_robot

In [ ]:
# YOLO 사진 탐지 클래스 
# 사용하려면 model_select를 객체화하고 run() 하고 result_list를 사용
class model_select:
    model = None
    result_list = None

    def __init__(self):
        self.model = torch.hub.load('ultralytics/yolov5', 'custom', path='/home/jwj/python_ws/duyeop/PinkLab-LobotArm-hackathon/best.pt', force_reload=True)
        self.model.conf = 0.8
    
    def get_image_from_webcam(self):
        """
        웹캠에서 이미지를 가져와 반환하는 함수
        """
        
        cap = cv2.VideoCapture(-1)
        if not cap.isOpened():
            raise ValueError("Could not open webcam")
        
        for i in range(20):
            
            # 카메라 초기화를 위해 2초 대기

            ret, frame = cap.read()
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            if not ret:
                raise ValueError("Could not read frame from webcam")
        
        cap.release()
        return frame

    def run_yolo(self, model, img_rgb):
        results = model(img_rgb)
        results.show()
        return np.array(results.xyxy[0].to('cpu'))
    
    def return_center_point(self, x1, x2, y1, y2):
        center_x = (x1 + x2)/2
        center_y = (y1 + y2)/2
        return center_x, center_y
    

    def get_yolo_results(self, results):
        result_list = []

        for i in results:
            if i[5] == 0:
                color = "yellow"
                shape = "Circle"
            elif i[5] == 1:
                color = "green"
                shape = "Rect"
            elif i[5] == 2:
                color = "red"
                shape = "Hexa"  

            center_x, center_y = self.return_center_point(i[0], i[2], i[1], i[3])
            result_list.append([color, shape, center_x, center_y])
        
        return result_list

    def run(self):
        img_rgb = self.get_image_from_webcam()
        results = self.run_yolo(self.model, img_rgb)
        self.result_list = self.get_yolo_results(results)

In [ ]:
# 로봇 동작과 관련한 함수를 모아놓은 코드

# 전역 변수 (현재 좌표)
current_actual = None

# PC-로봇 연결 함수 (TCP/IP 통신)
def connect_robot(ip):
    try:
        dashboard_p = 29999
        move_p = 30003
        feed_p = 30004
        print("연결 설정 중...")

        dashboard = DobotApiDashboard(ip, dashboard_p)
        move = DobotApiMove(ip, move_p)
        feed = DobotApi(ip, feed_p)
        print("연결 성공!!")

        return dashboard, move, feed

    except Exception as e:
        print(":연결 실패ㅜㅜ")
        raise e
        
# 로봇 에러 메시지 초기화 함수
def robot_clear(dashboard : DobotApiDashboard):
    dashboard.ClearError()

# 로봇 속도 조절 함수 (speed_value : 1~100)
def robot_speed(dashboard : DobotApiDashboard, speed_value):
    dashboard.SpeedFactor(speed_value)

# 그리퍼 구동 함수 (status 0 -> OFF, 1-> ON)
def gripper_DO(dashboard : DobotApiDashboard, index, status):
    dashboard.ToolDO(index, status)

# 현재 로봇 위치 받아오기 (로봇 베이스 좌표계 기준)
def get_Pose(dashboard : DobotApiDashboard):
    dashboard.GetPose()

# 로봇 구동 함수 (현재 위치 -> 목표 위치(point_list))
def run_point(move: DobotApiMove, point_list: list):
    move.MovL(point_list[0], point_list[1], point_list[2], point_list[3])
    
def get_feed(feed: DobotApi):
    global current_actual
    hasRead = 0

    while True:
        data = bytes()
        while hasRead < 1440:
            temp = feed.socket_dobot.recv(1440 - hasRead)
            if len(temp) > 0:
                hasRead += len(temp)
                data += temp
        hasRead = 0
        a = np.frombuffer(data, dtype=MyType)

        if hex((a['test_value'][0])) == '0x123456789abcdef':
            current_actual = a["tool_vector_actual"][0]     # Refresh Properties
        sleep(0.001)

# 카메라 좌표계와 로봇 좌표계를 변환하는 함수
def convert_center(center_x, center_y):
    ratio_x = 200/210
    ratio_y = 280/300

    # y
    if center_y >= 310:
        cam_leng_y = 460-center_y
        rb_leng_y = ratio_y * cam_leng_y
        rb_y = 140-rb_leng_y
    elif center_y < 310:
        cam_leng_y = 310-center_y
        rb_leng_y = ratio_y * cam_leng_y
        rb_y = -rb_leng_y
    # x
    if center_x >= 205:
        cam_leng_x = 310-center_x
        rb_leng_x = ratio_x * cam_leng_x
        rb_x = 380-rb_leng_x
    elif center_x < 205:
        cam_leng_x = 205-center_x
        rb_leng_x = ratio_x * cam_leng_x
        rb_x = 280-rb_leng_x
    
    return rb_x, rb_y

In [ ]:
### 입력 파라미터 #################################################################################
ip = "192.168.1.6"                  # Robot의 IP 주소
speed_value = 30                    # 로봇 속도 (1~100 사이의 값 입력)
gripper_port = 1                    # 그리퍼 포트 번호
   # 로봇의 초기 좌표 (x, y, z, yaw)

In [ ]:
# 로봇 연결
dashboard, move, feed = connect_robot(ip)
dashboard.EnableRobot()
print("이제 로봇을 사용할 수 있습니다!")

# 쓰레드 설정
feed_thread = threading.Thread(target=get_feed, args=(feed,))
feed_thread.setDaemon(True)
feed_thread.start()

In [ ]:
# 로봇 상태 초기화 1 : 로봇 에러 메시지 초기화
robot_clear(dashboard) 

# 로봇 상태 초기화 2 : 로봇 속도 조절
robot_speed(dashboard, speed_value)

# 로봇 현재 위치 받아오기 (x, y, z, yaw) - 로봇 베이스 좌표계 
get_Pose(dashboard)

In [ ]:
class point_vector():
    goal_point_red_temp = [[213.638470,-70.922574, 100, 0], [271.051270,-110.399032, 100, 0], [213.638470,-70.922574, 100, 0], [271.051270,-110.399032, 100, 0] ]
    goal_point_red = [[213.638470,-70.922574, -59.3, 0], [271.051270,-110.399032, -59.3, 0] ]

    goal_point_yellow_temp = [[208.608283,77.856783, 100, 0], [281.212810,115.661615, 100, 0], [208.608283,77.856783, 100, 0], [281.212810,115.661615, 100, 0]]
    goal_point_yellow = [[208.608283,77.856783, -59.3, 0], [281.212810,115.661615, -59.3, 0], [208.608283,77.856783, -59.3, 0], [281.212810,115.661615, -59.3, 0]]

    point_check = [246.085433,-103.949833, 100, 15]
    point_home = [223, -19, 100, 180]
    
    num_reds = 0
    num_yellow = 0
    num_green = 0

    afnum_reds = 0
    afnum_yellow = 0
    afnum_green = 0

    temp_5_floor = ""

    red_count = 0
    yellow_count = 0

In [ ]:
# 모델 객체화
Model = model_select()
point = point_vector()

In [ ]:
def length_6(floor:float):
    t = 0
    coordinate_1st_floor_list = []
    while (t < len(Model.result_list)):
        rb_x ,rb_y = convert_center(Model.result_list[t][3], Model.result_list[t][2])
        point_grip_temp = [rb_x, rb_y, 100, 15 ]
        point_grip = [rb_x, rb_y, floor, 15 ]
        print("목표지점" + str(point_grip))

        if Model.result_list[t][0] == "red" and point.red_count < 4:
            # 파라미터 수정 필요
            if Model.result_list[t][2] <= 218  and Model.result_list[t][3] <= 163:
                pass
            else:
                # 로봇 구동 1 (초기 위치)
                run_point(move, point.point_home)
                sleep(3)
                # 로봇 구동 3 (목표 위치)
                run_point(move, point_grip_temp)
                sleep(3)
                # 로봇 구동 3 (목표 위치)
                run_point(move, point_grip)
                gripper_DO(dashboard, gripper_port, 1)
                sleep(3)

                # 로봇 구동 3 (목표 위치)
                run_point(move, point_grip_temp)
                sleep(3)

                run_point(move, point.goal_point_red_temp[point.red_count])
                sleep(3)

                run_point(move, point.goal_point_red[point.red_count])
                gripper_DO(dashboard, gripper_port, 0)
                point.red_count += 1
                sleep(3)

                if floor == -59.3:
                    coordinate_1st_floor_list.append(point_grip)

        elif Model.result_list[t][0] == "yellow" and point.yellow_count < 4 :
            # 파라미터 수정 필요
            if Model.result_list[t][2] <= 400 and  Model.result_list[t][3] <= 150:
                pass
            else:
                # 로봇 구동 1 (초기 위치)
                run_point(move, point.point_home)
                sleep(3)

                # 로봇 구동 3 (목표 위치)
                run_point(move, point_grip_temp)
                sleep(3)

                # 로봇 구동 3 (목표 위치)
                run_point(move, point_grip)
                gripper_DO(dashboard, gripper_port, 1)
                sleep(3)

                # 로봇 구동 3 (목표 위치)
                run_point(move, point_grip_temp)
                sleep(3)

                run_point(move, point.goal_point_yellow_temp[point.yellow_count])
                sleep(3)

                run_point(move, point.goal_point_yellow[point.yellow_count])
                gripper_DO(dashboard, gripper_port, 0)
                point.yellow_count += 1
                sleep(3)

                if floor == -59.3:
                    coordinate_1st_floor_list.append(point_grip)
        t += 1

    return coordinate_1st_floor_list

In [ ]:
def length_5():
    for i in Model.result_list:
        if i[0] == 'red':
            point.num_reds += 1
        elif i[0] == 'yellow':
            point.num_yellow += 1
        elif i[0] == 'green':
            point.num_green += 1

    run_point(move, point.point_home)
    sleep(3)
    # 돌아돌아
    for j in Model.result_list:
        rb_x ,rb_y = convert_center(j[3], j[2])
        point_grip_temp = [rb_x, rb_y, 100, 15 ]
        point_grip = [rb_x, rb_y, -42.5, 15]

        run_point(move, point_grip)
        gripper_DO(dashboard, gripper_port, 1)
        sleep(3)


    run_point(move, point.point_home)
    sleep(3)

    Model.run()
    for i in Model.result_list:
        if i[0] == 'red':
            point.afnum_reds += 1
        elif i[0] == 'yellow':
            point.afnum_yellow += 1
        elif i[0] == 'green':
            point.afnum_green += 1

    if point.num_reds - point.afnum_reds == 0:
        pass
    elif point.num_reds - point.afnum_reds == 1:
        temp_5_floor = "r"

    if point.num_yellow - point.afnum_yellow == 0:
        pass
    elif point.num_yellow - point.afnum_yellow == 1:
        temp_5_floor = "y"

    if point.num_green - point.afnum_green == 0:
        pass
    elif point.num_green - point.afnum_green == 1:
        temp_5_floor = "g"

    
    if temp_5_floor == "r":
        run_point(move, point.goal_point_red_temp[point.red_count])
        sleep(3)

        run_point(move, point.goal_point_red[point.red_count])
        gripper_DO(dashboard, gripper_port, 0)
        point.red_count += 1
        sleep(3)
        
    elif temp_5_floor == "g":
        for i in Model.result_list:
            if i[0] == "green":
                rb_x ,rb_y = convert_center(i[3], i[2])
                point_grip_temp = [rb_x, rb_y, 100, 15 ]
                point_grip = [rb_x, rb_y, -41, 15 ]

                run_point(move, point_grip_temp)
                sleep(3)

                run_point(move, point_grip)
                gripper_DO(dashboard, gripper_port, 0)


    elif temp_5_floor == "y":
        run_point(move, point.goal_point_yellow_temp[point.yellow_count])
        sleep(3)

        run_point(move, point.goal_point_yellow[point.yellow_count])
        gripper_DO(dashboard, gripper_port, 0)
        point.yellow_count += 1
        sleep(3)


    print("남은 아이들을 옮깁니다.!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! \n\n\n주의 이부분에서 비상버튼 누르세요")


    length_6(-59.3)

    

4개 이하 알고리즘 접근 방법.


초록색이 몇개이냐가 중요한 포인트


1. 일단 6개 알고리즘으로 2층이라 생각하고 하세요. (초록색은 제외합니다.)

2. 6개 알고리즘으로 1층이라 생각하고 하세요. (초록색은 제외합니다.)
(이때 이동하는 좌표를 암기합니다. A좌표와 B좌표가 구해집니다.)

3. 총 이동이 4번이면 종료합니다. 
4. 총 이동이 3번일때 
	(3번 이하면 2층에 초록색이 가리고 있는 상태이며, 카메라상에서 초록만 남습니다.)
	2층이라 생각하고 초록을 A좌표(다른 초록은 B로 이동)로 이동시킵니다. 	
	스캔을 시작합니다. 초록이 가리고 있는게 치워집니다.
	1층이라 생각하고 알고리즘을 적용합니다.

5. 총 이동이 2번일때
	초록의 아래에 두 개가 숨어 있는 상태입니다. 
	2층이라 생각하고 초록을 A좌표로 이동하고, 초록을 B좌표로 이동합니다.
	스캔을 시작합니다.
	1층이라 생각하고 알고리즘을 적용합니다.



3개 이하 알고리즘
1. 2층이라 생각하고 초록을 제외하고 움직입니다.
2. 총 이동이 4번이면 종료합니다.
3. 총 이동이 3번이면 위와 같습니다.
4. 총 이동이 2번이면 위와 같습니다.

즉 4개 이하 알고리즘은 동일합니다.



In [1]:
def length_under_4_3():
    back = 0
    R_y = 0

    # 우선 2층이라 생각하고 옮깁니다.
    length_6(-42.5)

    # 옮긴 상태에서 다시 detection합니다.
    # 모두 1층만 남을 것입니다.
    Model.run()
    
    # 2층이라 생각하고 옮깁니다. 좌표를 받아옵니다.
    coordinate_1st_floor_list = length_6(-59.3)

    move_num_reds = 0
    move_num_yellow = 0
    #다시 스캔을 시작합니다.
    Model.run()
    
    t = 0
    second_list = []
    while (t < len(Model.result_list)):
        if Model.result_list[t][0] == "red" and point.red_count < 2:
            # 파라미터 수정 필요
            if Model.result_list[t][2] <= 218  and Model.result_list[t][3] <= 163:
                move_num_reds += 1
            else:
                second_list.append(Model.result_list[t])

        elif Model.result_list[t][0] == "yellow" and point.yellow_count < 2 :
            # 파라미터 수정 필요
            if Model.result_list[t][2] <= 400 and  Model.result_list[t][3] <= 150:
                move_num_yellow += 1
            else:
                second_list.append(Model.result_list[t])


    if move_num_reds + move_num_yellow == 3:
        rb_x ,rb_y = convert_center(second_list[0][3], second_list[0][2])
        point_grip_temp1 = [rb_x, rb_y, 100, 15 ]
        point_grip1 = [rb_x, rb_y, -42.5, 15]

        rb_x ,rb_y = convert_center(second_list[1][3], second_list[1][2])
        point_grip_temp2 = [rb_x, rb_y, 100, 15 ]
        point_grip2 = [rb_x, rb_y, -42.5, 15]

        run_point(move, point_grip_temp1)
        gripper_DO(dashboard, gripper_port, 1)
        sleep(1)

        run_point(move, point_grip1)
        gripper_DO(dashboard, gripper_port, 1)
        sleep(1)

        run_point(move, point.point_home)
        sleep(1)

        run_point(move, point.point_check)
        sleep(5)

        Model.run()

        for i in Model.result_list:
            if i[0] == 'red':
                #수치 조정 필요합니다.
                if i[2] <= 218  and i[3] <= 163:
                    pass
                else:
                    back = 1
                    R_y = 0
                    run_point(move, point_grip_temp2)
                    gripper_DO(dashboard, gripper_port, 1)
                    sleep(1)

                    run_point(move, point_grip2)
                    gripper_DO(dashboard, gripper_port, 1)
                    sleep(1)

                    run_point(move, point.point_home)
                    sleep(1)

            elif i[0] == 'yellow':
                #수치 조정 필요합니다.
                if i[2] <= 400 and  i[3] <= 150:
                    pass
                else:
                    back = 1
                    R_y = 1
                    run_point(move, point_grip_temp2)
                    gripper_DO(dashboard, gripper_port, 1)
                    sleep(1)

                    run_point(move, point_grip2)
                    gripper_DO(dashboard, gripper_port, 1)
                    sleep(1)

                    run_point(move, point.point_home)
                    sleep(1)
            
            if back == 1:
                rb_x ,rb_y = convert_center(second_list[0][3], second_list[0][2])
                point_grip_temp= [rb_x, rb_y, 100, 15 ]
                point_grip = [rb_x, rb_y, -42.5, 15]

                run_point(move, point_grip_temp1)
                gripper_DO(dashboard, gripper_port, 1)
                sleep(1)

                run_point(move, point_grip1)
                gripper_DO(dashboard, gripper_port, 1)
                sleep(1)

                run_point(move, point.point_home)
                sleep(1)

                if R_y == 0:
                    run_point(move, point.goal_point_red_temp[point.red_count])
                    sleep(3)

                    run_point(move, point.goal_point_red[point.red_count])
                    gripper_DO(dashboard, gripper_port, 0)
                    sleep(3)

                if R_y == 1:
                    run_point(move, point.goal_point_yellow_temp[point.yellow_count])
                    sleep(3)

                    run_point(move, point.goal_point_yellow[point.yellow_count])
                    gripper_DO(dashboard, gripper_port, 0)
                    sleep(3)

                run_point(move, point.point_home)
                sleep(1)

                return



        rb_x ,rb_y = convert_center(second_list[0][3], second_list[0][2])
        point_grip_temp1 = [rb_x, rb_y, 100, 15 ]
        point_grip1 = [rb_x, rb_y, -42.5, 15]

        rb_x ,rb_y = convert_center(second_list[1][3], second_list[1][2])
        point_grip_temp2 = [rb_x, rb_y, 100, 15 ]
        point_grip2 = [rb_x, rb_y, -42.5, 15]

        run_point(move, point_grip_temp2)
        gripper_DO(dashboard, gripper_port, 1)
        sleep(1)

        run_point(move, point_grip2)
        gripper_DO(dashboard, gripper_port, 1)
        sleep(1)

        run_point(move, point.point_home)
        sleep(1)

        run_point(move, point.point_check)
        sleep(5)

        Model.run()

        for i in Model.result_list:
            if i[0] == 'red':
                #수치 조정 필요합니다.
                if i[2] <= 218  and i[3] <= 163:
                    pass
                else:
                    back = 1
                    R_y = 0
                    run_point(move, point_grip_temp1)
                    gripper_DO(dashboard, gripper_port, 1)
                    sleep(1)

                    run_point(move, point_grip1)
                    gripper_DO(dashboard, gripper_port, 1)
                    sleep(1)

                    run_point(move, point.point_home)
                    sleep(1)

            elif i[0] == 'yellow':
                #수치 조정 필요합니다.
                if i[2] <= 400 and  i[3] <= 150:
                    pass
                else:
                    back = 1
                    R_y = 0
                    run_point(move, point_grip_temp2)
                    gripper_DO(dashboard, gripper_port, 1)
                    sleep(1)

                    run_point(move, point_grip2)
                    gripper_DO(dashboard, gripper_port, 1)
                    sleep(1)

                    run_point(move, point.point_home)
                    sleep(1)
            
            if back == 1:
                rb_x ,rb_y = convert_center(second_list[0][3], second_list[0][2])
                point_grip_temp= [rb_x, rb_y, 100, 15 ]
                point_grip = [rb_x, rb_y, -42.5, 15]

                run_point(move, point_grip_temp1)
                gripper_DO(dashboard, gripper_port, 1)
                sleep(1)

                run_point(move, point_grip1)
                gripper_DO(dashboard, gripper_port, 1)
                sleep(1)

                run_point(move, point.point_home)
                sleep(1)

                if R_y == 0:
                    run_point(move, point.goal_point_red_temp[point.red_count])
                    sleep(3)

                    run_point(move, point.goal_point_red[point.red_count])
                    gripper_DO(dashboard, gripper_port, 0)
                    sleep(3)

                if R_y == 1:
                    run_point(move, point.goal_point_yellow_temp[point.yellow_count])
                    sleep(3)

                    run_point(move, point.goal_point_yellow[point.yellow_count])
                    gripper_DO(dashboard, gripper_port, 0)
                    sleep(3)

                run_point(move, point.point_home)
                sleep(1)

                return

            
                    

    

    
    elif move_num_reds + move_num_yellow == 2:
        t = 0
        count = 0
        while (t < len(second_list)):
            point_grip_temp = [coordinate_1st_floor_list[count][0], coordinate_1st_floor_list[count][1], 100, 15 ]
            point_grip =[coordinate_1st_floor_list[count][0], coordinate_1st_floor_list[count][1], -45.5, 15 ]

            if count == 0 and second_list[0] == "green":
                # 로봇 구동 1 (초기 위치)
                run_point(move, point.point_home)
                sleep(3)
                # 로봇 구동 3 (목표 위치)
                run_point(move, point_grip_temp)
                sleep(3)
                # 로봇 구동 3 (목표 위치)
                run_point(move, point_grip)
                gripper_DO(dashboard, gripper_port, 1)
                sleep(3)

                # 로봇 구동 3 (목표 위치)
                run_point(move, point_grip_temp)
                sleep(3)

                run_point(move, point.goal_point_red_temp[point.red_count])
                sleep(3)

                run_point(move, point.goal_point_red[point.red_count])
                gripper_DO(dashboard, gripper_port, 0)
                sleep(3)
                count += 1

            elif count == 1 and second_list[0] == "green":
                # 로봇 구동 1 (초기 위치)
                run_point(move, point.point_home)
                sleep(3)
                # 로봇 구동 3 (목표 위치)
                run_point(move, point_grip_temp)
                sleep(3)
                # 로봇 구동 3 (목표 위치)
                run_point(move, point_grip)
                gripper_DO(dashboard, gripper_port, 1)
                sleep(3)

                # 로봇 구동 3 (목표 위치)
                run_point(move, point_grip_temp)
                sleep(3)

                run_point(move, point.goal_point_red_temp[point.red_count])
                sleep(3)

                run_point(move, point.goal_point_red[point.red_count])
                gripper_DO(dashboard, gripper_port, 0)
                sleep(3)
                count += 1

        # 1층을 다시 합니다.
        length_6(-59.3)


In [ ]:
# 정답?
Model.run()
if len(Model.result_list) == 6:
    length_6()
elif len(Model.result_list) == 5:
    length_5()

elif len(Model.result_list) == 5 or  len(Model.result_list) == 4:
    length_under_4_3()
